In [1]:
import pandas as pd, time, requests, json
from bs4 import BeautifulSoup
import numpy as np

# Stage 1

In [2]:
import os
from os.path import isfile, join
import shutil
mypath='profiles/'
profiles = [f[:-5] for f in os.listdir(mypath) if isfile(join(mypath, f))]
missing=[]

In [3]:
nodes={}
for p in profiles:
  try:
    profile=json.load(open(mypath+p+'.json','r'))
    if 'code' in profile:
      if profile['code'] in [404]:
        print(p,profile['code']) #delete these files
    else:
      nodes[p]=profile
  except:
    print(p)
    missing.append(p)

l%C5%91rinc-bo%C3%A9r-a706a9188 404


In [4]:
open('missing.json','w').write(json.dumps(missing))

2

Go back to parser, and try again `missing`

In [9]:
#update master file
profiles = [f[:-5] for f in os.listdir(mypath) if isfile(join(mypath, f))]

In [46]:
#resolve missing
mi=pd.read_excel('mi_clean.xlsx')
for i in mi['link'].values:
  if str(i)!='nan':
    url=i.split('?')[0]
    link=url.split('/')[4]
    if link not in profiles:
      print(i,link)

In [51]:
len(mi['link'].dropna()),len(mi['link'].dropna().drop_duplicates())

(786, 785)

In [52]:
#resolve duplicates
mi.dropna()[mi['link'].dropna().duplicated()]

Contact Id Family Name First Name        Full Name  \
878  0031v00002gpF85AAE       Szabó  Krisztina  Szabó Krisztina   

     GDPR Consent? Email (if GDPR accepted)  Year of Graduation  \
878          False               No consent                2013   

                                     Final Destination  Good  Round  \
878  University of Sussex - United Kingdom - Biomed...  50.0    2.0   

                                                  link  
878  https://www.linkedin.com/in/krisztina-rita-szabo/

In [54]:
mi[mi['link']=='https://www.linkedin.com/in/krisztina-rita-szabo/'] #seems legit

Contact Id Family Name First Name        Full Name  \
877  0031v00002Y0vFIAAZ       Szabó  Krisztina  Szabó Krisztina   
878  0031v00002gpF85AAE       Szabó  Krisztina  Szabó Krisztina   

     GDPR Consent? Email (if GDPR accepted)  Year of Graduation  \
877          False               No consent                2012   
878          False               No consent                2013   

                                     Final Destination  Good  Round  \
877  University of Sussex - United Kingdom - Biomed...  60.0    2.0   
878  University of Sussex - United Kingdom - Biomed...  50.0    2.0   

                                                  link  
877  https://www.linkedin.com/in/krisztina-rita-szabo/  
878  https://www.linkedin.com/in/krisztina-rita-szabo/

In [63]:
mi['public_identifier']=mi['link'].str.split('?').str[0].str.split('/').str[4]

In [64]:
mi.to_excel('mi_clean.xlsx')

Languages

In [55]:
for i in nodes:
    node=nodes[i]
    if 'languages' in node:
        node['languages']=[j.split(' ')[0].replace(',','').strip()\
        .replace('Bokmål','Norwegian').replace('Deutsch','German').replace('Italien','Italian')\
        .replace('Nederlands','Dutch').replace('Walser','German') for j in node['languages']]

In [56]:
languages=set([j for i in pd.DataFrame(nodes).T['languages'].values for j in i])

Skills

In [57]:
for i in nodes:
    node=nodes[i]
    if 'skills' in node:
        node['skills']=[j.replace(',','').replace(' skills','').strip()\
        .replace('accountinging','accounting').replace('dimension','dimensions')\
        .replace('english as a second language','english')\
        .replace("enseignement de l'anglais",'english') for j in node['skills']]
        for skill in node['skills']:
            if skill.title() in languages:
                if 'languages' in node:
                    if skill.title() not in node['languages']:
                        node['languages'].append(skill.title())
                else:
                    node['languages']=[skill.title()]
        node['skills']=[i for i in node['skills'] if i not in languages]



In [58]:
skills=set([j for i in pd.DataFrame(nodes).T['skills'].values for j in i])

In [59]:
open('nodes.json','w').write(json.dumps(nodes))

7154197